In [ ]:
from quspin.operators import hamiltonian
from quspin.basis import spin_basis_1d
from quspin.tools.measurements import ent_entropy, diag_ensemble
from numpy.random import ranf,seed
from joblib import delayed,Parallel
import pyqentangle
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import svd
import math
def is_not_number(x):
    return not isinstance(x, (int, float))
L = 12
sizeA = 4
sizeB = 8
basis = spin_basis_1d(L,pauli=False)

def generate_Hamiltonian(v,g,norm = np.sqrt(2),J=1,h=1):
    #Used here the normalization that sqrt(u^2 + v^2) = sqrt(2)
    u = np.sqrt(norm**2-v**2)

    #Defining the coefficients for coupling strengths
    Jx =  J*((u+v)/2)
    Jy = J*((v-u)/2)
    Jz = J*v

    #Nearest-neighbour coupling lists
    J_xx = [[Jx, i,i+1] for i in range(L-1)]
    J_yy = [[Jy, i, i+1] for i in range(L-1)]
    J_zz = [[Jz, i, i+1] for i in range(L-1)]

    #Local field drawn from a uniform random sample
    h_z = np.random.uniform(-h, h, size=L)
    h_x = np.random.uniform(-h, h, size=L)
    disordered_z_field = [[g * h_z[i], i] for i in range(L)]
    disordered_x_field = [[g * h_x[i], i] for i in range(L)]

    static = [["xx",J_xx], ["yy",J_yy], ["zz", J_zz], ["x", disordered_x_field], ["z", disordered_z_field]]
    dynamic = []
    no_checks = {"check_symm": False, "check_herm": False, "check_pcon": False}
    return hamiltonian(static, dynamic, basis = basis, dtype = np.float64, **no_checks)


def EES(hamiltonian, sizeA, sizeB):
    E,V=hamiltonian.eigh()
    all_ratios = []

    #Looping through every eigenstate to get the corresponding ratios
    for eigenstate in V[int(len(V)/3):2*int(len(V)/3) + 1]: 
        ratios_list = ratios_for_an_eigenstate(eigenstate, sizeA, sizeB)

        #Then adding the ratios collected for every eigenstate to the main list
        for ratio in ratios_list:
            all_ratios.append(ratio)
    
    #Constructing a histogram of all the ratios
    counts, bin_edges = np.histogram(all_ratios, bins=180, density=True)
    bin_width = np.diff(bin_edges)
    x_values = (bin_edges[:-1] + bin_edges[1:]) / 2
    y_values = counts

    
    return x_values, y_values

def ratios_for_an_eigenstate(eigenstate, sizeA, sizeB):
    zetas = get_zetas(eigenstate, sizeA, sizeB) #Getting the zeta values first
    ratios_list = []

    k = 1
    while (k<=(len(zetas)-2)):
        s_k = zetas[k+1]-zetas[k]
        s_k_1 = zetas[k]-zetas[k-1]
        differences = [s_k,s_k_1]

        if(np.max(differences) > 10**(-200)): #Somehow this fixed the issue with NaN values popping up in the ratios
            r = np.min(differences) / np.max(differences)
            ratios_list.append(r)
        
        k += 1
    
    return ratios_list
    
def get_zetas(eigenstate, sizeA, sizeB):
    zetas = []
    schmidt = schmidt_decomposition(eigenstate, sizeA,sizeB)

    #Calcutlaing the zeta values only if we have > 2 schmidt coeffecients (Need at least 3 values to calculate one ratio)
    if(np.count_nonzero(schmidt)>2): 
        for coeff in schmidt:
            if coeff != 0: #Avoiding math errors
                zetas.append(-2*np.log(coeff))
    return zetas

def schmidt_decomposition(state_vector, sizeA, sizeB):
    state_matrix = state_vector.reshape(2**sizeA, 2**sizeB)
    U, S, Vh = svd(state_matrix)
    schmidt_coefficients = S
    basis_A = U
    basis_B = Vh.conj().T
    return schmidt_coefficients

In [ ]:
from scipy.optimize import curve_fit

#Functions to fit the data collected from the histograms
def semi_poisson(x):
    return 6 * x/(1+x)**4

def goe(x):
    return (27/8) * (x+x**2)/(1+x+x**2)**(2.5)

def linear_combination(x,a,b):
    return a * semi_poisson(x) + b * goe(x)

def fit_distribution(hamiltonian, sizeA, sizeB):
    x_values, y_values = EES(hamiltonian, sizeA, sizeB)
    popt, pcov = curve_fit(linear_combination, x_values, y_values)
    a, b = popt
    return a,b, popt

def plot_data_fit(hamiltonian, sizeA, sizeB):
    x_values, y_values = EES(hamiltonian, sizeA, sizeB)
    a,b, popt = fit_distribution(hamiltonian, sizeA, sizeB)
    plt.scatter(x_values, y_values, color='white', edgecolor='blue')
    plt.plot(x_values, linear_combination(x_values, *popt), color='red', label='Fit: af(x) + bg(x)')
    plt.show()

#Example
H = generate_Hamiltonian(0,0)
plot_data_fit(H, sizeA, sizeB)
a,b, popt = fit_distribution(H, sizeA, sizeB)
print(a,b)

In [ ]:
#Plotting out a colormap to see the changes of the fit strength of semi-poisson/goe as we vary the interaction strength and/or local field
V = np.arange(0, 1, 0.1) 
G = np.arange(0, 9, 0.1)

which_fit = "goe" #Change it to "semi_poisson" for semi_poisson fit strength

#Taking values of v, g and constructing the corresponding hamiltonian
def calculate_intensity(v, g, sizeA, sizeB):
    H = generate_Hamiltonian(v,g)
    #Performs EES first to get the ratios and then fits the data to a linear combination of semi-poisson and GOE
    a,b, popt = fit_distribution(H, sizeA, sizeB) 
    #a corresponds to semi-poisson fit and b corresponds to GOE/semi-poisson fit
    if (which_fit == "goe"):
        return np.absolute(b)/np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    elif (which_fit == "semi_poisson"):
        return np.absolute(a)/np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    #Different normalization here than the one kent used to limit the strength to the range (0,1)

def intensity_mesh(V,G, sizeA, sizeB):
    intensity = np.zeros((len(G), len(V)))
    for i in range(len(G)):
        for j in range(len(V)):
            intensity[i, j] = calculate_intensity(V[j], G[i], sizeA,sizeB)
        print(f'{(G[i]/8.9)*100}% done')
    return intensity

A = intensity_mesh(V, G, sizeA, sizeB)
V_mesh, G_mesh = np.meshgrid(V, G)

plt.figure(figsize=(8, 6))
plt.pcolormesh(V_mesh, G_mesh, A, shading="auto", cmap='viridis')
plt.colorbar(label='b')
plt.xlabel('v')
plt.ylabel('g')
plt.title(f'{which_fit} fit strength for {sizeA}-{sizeB} partition')
plt.show()